## Installing tensorflow in your vm

You can pretty easily add tensorflow to your vm to support some small deep learning projects.  Follow the instructions below.

First open a terminal in your vm and do 

```
sudo apt-get update
sudo apt-get upgrade
```

Next, start your virtual environment.  Mine is called "test" so I start with:

```
source test/bin/activate
```

Next, install tensorflow:

```
pip install --upgrade tensorflow
```

Test your installation with the following code:

In [32]:
import tensorflow as tf
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [33]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


## Computational Graphs and Sessions

OK, now we have tf installed we can try to do something interesting with it.  But first we need to understand a little about the execution model and what happens internally with tensorflow.

Tensorflow uses graphs to define functions to execute.  Each constant, variable, and expression is  node of the graph.  This provides a sometimes complex but very flexible method for defining and executing functions.  Some of the important benefits are:

* Graphs can be evaluated starting at any node.  Evaluating a node implie evaluating all previous nodes.
* Parallel computations are easier to figure out since dependent relationships are explicitly determined by the graph.

Below is a very simple example of a tesorflow computational graph:

In [34]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape  # m= number of samples, n = number of predictors
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

In [35]:
reset_graph()

# Question: what learning method is implemented below?
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [36]:
theta_value, n

(array([[ -3.74651413e+01],
        [  4.35734153e-01],
        [  9.33829229e-03],
        [ -1.06622010e-01],
        [  6.44106984e-01],
        [ -4.25131839e-06],
        [ -3.77322501e-03],
        [ -4.26648885e-01],
        [ -4.40514028e-01]], dtype=float32), 8)

In the above setup, we see that the computational graph is evaluated on an object called a "session".

Below we have an example of a neural network with a single hidden layer that solves a classifier for the iris data set.  This is using some high-level tools provided byt tensorflow.

In [37]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


# import some data to play with
iris = load_iris()
#X = iris.data[:, :2]  # we only take the first two features.
X = iris.data  # we only take the first two features.
Y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

X_train.shape, y_train.shape

((120, 4), (120,))

In [38]:
reset_graph()

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
net = tf.contrib.learn.DNNClassifier(hidden_units=[6,6], n_classes=3, feature_columns=feature_cols)
net = tf.contrib.learn.SKCompat(net)
net.fit(X_train, y_train, batch_size=20, steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_evaluation_master': '', '_session_config': None, '_tf_random_seed': None, '_num_worker_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_master': '', '_environment': 'local', '_task_type': None, '_is_chief': True, '_save_summary_steps': 100, '_model_dir': '/tmp/tmp5nt2iic1', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f50f0e0b8>, '_task_id': 0}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp5nt2iic1/model.ckpt.
INFO:tensorflow:loss = 1.3551, step = 1
INFO:tensorflow:global_step/sec: 382.067
INFO:tensorflow:loss = 0.160398, step = 101 (0.269 sec)
INFO:tensorflow:global_step/sec: 501.38
INFO:tensorflow:loss = 0.

SKCompat()

In [39]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_predict = net.predict(X_test)
confusion_matrix(y_test, y_predict['classes'])

INFO:tensorflow:Restoring parameters from /tmp/tmp5nt2iic1/model.ckpt-1000


array([[11,  0,  0],
       [ 0, 13,  0],
       [ 0,  0,  6]])

In [40]:
print(classification_report(y_test, y_predict['classes']))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        11
          1       1.00      1.00      1.00        13
          2       1.00      1.00      1.00         6

avg / total       1.00      1.00      1.00        30



### A Little More
Tensorflow is really involved.  The example below is designed to solve for MNIST using tensorflow but is a more manually constructed network.

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/MNIST/", one_hot=True)

n_inputs = 28*28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [1]:
import numpy as np
import tensorflow as tf
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
reset_graph()

# The input data X for training and prediction as well 
#   as the true y values for training are added as placeholders 
#   because 
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y") 

# We use a named scope to define the network portion of the 
#  computational graph
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    
# Another named scope for the loss function portion of the graph
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar('log_loss', loss)
    
# Another for the training steps
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
# and finally a named scope for evaluating the model against test data
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)
    
# set up a node for initializing the variables,
init = tf.global_variables_initializer()

# and a node for saving the model after training
saver = tf.train.Saver()

# Create a local folder called 'tf_log' for the logs, define 
#   the format for the log folder for each training run.
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

logdir = log_dir("mnist_dnn")
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

NameError: name 'n_inputs' is not defined

In [43]:
import os
# now lets run this graph to train it
m, n = X_train.shape

n_epochs = 10001
batch_size = 50
n_batches = int(np.ceil(m / batch_size))

checkpoint_path = "/tmp/my_deep_mnist_model.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./my_deep_mnist_model"

best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress = 50

with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        # if the checkpoint file exists, restore the model and load the epoch number
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)

    for epoch in range(start_epoch, n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})


InvalidArgumentError: labels must be 1-D, but got shape [50,10]
	 [[Node: loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](dnn/outputs/BiasAdd, _arg_y_0_1)]]

Caused by op 'loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/joe/test/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/joe/test/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/joe/test/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/ho